In [1]:
from utils import *
from output_formatter import transform_to_prodigy
from nervaluate import Evaluator

In [2]:
model = get_base_model()
tokenizer = get_base_tokenizer()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
sentences = []
true_responses = []
pred_responses = []

In [4]:
with open('data/CDR_TestSet.json', 'r') as file:
    data = json.load(file)

for item in data:
    if 'user' not in item or 'assistant' not in item:
        print("problem with item:", item)
    sentences.append(item['user'])
    true_responses.append(transform_to_prodigy(item['user'], item['assistant']))

In [6]:
for i in range(10):
    response = generate_response(sentences[i], model, tokenizer)
    pred_responses.append(transform_to_prodigy(sentences[i], response))

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48
You are not running the flash-attention implementation, expect numerical differences.


In [9]:
pred_responses[0], true_responses[0], sentences[0]

([{'start': 23, 'end': 31, 'label': 'Disease'},
  {'start': 1, 'end': 11, 'label': 'Chemical'},
  {'start': 157, 'end': 163, 'label': 'Disease'}],
 [{'start': 1, 'end': 11, 'label': 'Chemical'},
  {'start': 23, 'end': 31, 'label': 'Disease'},
  {'start': 56, 'end': 66, 'label': 'Chemical'},
  {'start': 157, 'end': 163, 'label': 'Disease'}],
 ' Famotidine-associated delirium. A series of six cases. Famotidine is a histamine H2-receptor antagonist used in inpatient settings for prevention of stress ulcers and is showing increasing popularity because of its low cost.')

In [12]:
evaluator = Evaluator(true_responses[:10], pred_responses, tags=['Disease', 'Chemical'])
results, results_per_tag, result_indices, result_indices_by_tag = evaluator.evaluate()
results

{'ent_type': {'correct': 31, 'incorrect': 2, 'partial': 0, 'missed': 19, 'spurious': 8, 'possible': 52, 'actual': 41, 'precision': 0.7560975609756098, 'recall': 0.5961538461538461, 'f1': 0.6666666666666667}, 'partial': {'correct': 31, 'incorrect': 0, 'partial': 2, 'missed': 19, 'spurious': 8, 'possible': 52, 'actual': 41, 'precision': 0.7804878048780488, 'recall': 0.6153846153846154, 'f1': 0.6881720430107527}, 'strict': {'correct': 30, 'incorrect': 3, 'partial': 0, 'missed': 19, 'spurious': 8, 'possible': 52, 'actual': 41, 'precision': 0.7317073170731707, 'recall': 0.5769230769230769, 'f1': 0.6451612903225806}, 'exact': {'correct': 31, 'incorrect': 2, 'partial': 0, 'missed': 19, 'spurious': 8, 'possible': 52, 'actual': 41, 'precision': 0.7560975609756098, 'recall': 0.5961538461538461, 'f1': 0.6666666666666667}}
